In [1]:
### here we import various libraries needed
import pandas as pd
import numpy as np
from ebmdatalab import bq
from ebmdatalab import charts
from ebmdatalab import maps

In [2]:
sql='''SELECT
presc.month,
pct,
ccg.org_type,
ccg.name,
presc.bnf_code,
presc.bnf_name,
quantity,
SUM(items) AS totalitems,
SUM(actual_cost) AS totalcost
FROM
ebmdatalab.hscic.normalised_prescribing_standard AS presc
JOIN
hscic.ccgs AS ccg 
ON
presc.pct = ccg.code
LEFT JOIN
ebmdatalab.dmd.product AS dmd
ON
presc.bnf_code=dmd.bnf_code
WHERE
dmd.is_blacklisted = true
GROUP BY
month,
pct,
ccg.org_type,
ccg.name,
presc.bnf_code,
presc.bnf_name,
quantity'''


df_blacklist = bq.cached_read(sql, csv_path='df_blacklist.csv', use_cache=True)  # add `use_cache=False` to override
df_blacklist['month'] = df_blacklist['month'].astype('datetime64[ns]')
df_blacklist.head()

,month,pct,org_type,name,bnf_code,bnf_name,totalitems,totalcost
0,2016-10-01,08L,CCG,NHS LEWISHAM CCG,1310020N0BBABAB,Daktarin_Pdr 2%,1,4.79000
1,2017-06-01,08N,CCG,NHS REDBRIDGE CCG,0904010J0BLADAK,Doves Farm_G/F Wte Plain Flour,3,49.46364
2,2013-05-01,02F,CCG,NHS WEST CHESHIRE CCG,0904010ACBIACAC,Juvela_G/F W/FSpecial Flakes Red Berries,1,6.32000
3,2015-09-01,02M,CCG,NHS FYLDE AND WYRE CCG,090401080BPABBD,Barkat_G/F W/F Buckwheat Penne,3,24.42000
4,2010-11-01,99A,CCG,NHS LIVERPOOL CCG,090401080BFACBD,Orgran_G/F W/F Buckwheat Spiral,3,24.60000


In [17]:
## ensuring the format is consistent for pounds and pence
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [18]:
### grouping the total cost by product
totalcost = df_blacklist.groupby(['bnf_name'])['totalcost'].sum()
totalcost.sort_values(ascending = False)

bnf_name
Juvela_G/F Pizza Base                      4886593.34
Glutafin_G/F W/F Spiral                    3047528.63
Glutafin_G/F W/F Pizza Base                2573821.48
Juvela_G/F W/F Fusilli                     2425069.24
Glutafin_G/F W/F Cracker                   1574435.75
Glutafin_G/F W/F Spaghetti                 1535744.97
Glutafin_G/F W/F Macaroni Penne            1336839.56
Juvela_G/F W/F Spaghetti                   1234030.76
Glutafin_G/F W/F Crisp Bread               1017253.61
Glutafin_G/F W/F Diges Bisc                 978959.28
Juvela_G/F Crispbread                       965679.22
Glutafin_G/F W/F Pasta Shell                907448.15
Glutafin_G/F W/F H/Fbre Cracker             721688.97
Juvela_G/F W/F Fibre Flakes                 679469.29
Juvela_G/F Tea Bisc                         671812.88
Glutafin_G/F W/F Shortbread Bisc            612724.72
Juvela_G/F W/F Fibre Penne                  601119.25
Juvela_G/F W/F Pure Oats                    577397.08
Juvela_G/F Pizza Ba

In [19]:
### grouping the total cost by CCG
totalcost = df_blacklist.groupby(['name'])['totalcost'].sum()
totalcost.sort_values(ascending = False)

name
NHS BIRMINGHAM AND SOLIHULL CCG                             1308535.63
NHS BRISTOL, NORTH SOMERSET AND SOUTH GLOUCESTERSHIRE CCG   1084260.04
NHS NORTHERN, EASTERN AND WESTERN DEVON CCG                 1067852.87
NHS LEEDS CCG                                                947556.60
NHS DORSET CCG                                               945462.17
NHS SHEFFIELD CCG                                            787281.81
NHS SOUTHERN DERBYSHIRE CCG                                  665237.17
NHS WILTSHIRE CCG                                            644684.86
NHS WEST HAMPSHIRE CCG                                       620101.54
NHS EAST AND NORTH HERTFORDSHIRE CCG                         604911.98
NHS COASTAL WEST SUSSEX CCG                                  583021.39
NHS GLOUCESTERSHIRE CCG                                      551635.54
NHS SOMERSET CCG                                             548373.81
NHS LIVERPOOL CCG                                            518927.41
N

In [20]:
### see impact of NCSO in 2018
bl2018 = df_blacklist.loc[(df_blacklist["month"]>="2018-01-01") & (df_blacklist["month"]<="2018-12-01")]
bl2018

,month,pct,org_type,name,bnf_code,bnf_name,totalitems,totalcost
25,2018-07-01,06A,CCG,NHS WOLVERHAMPTON CCG,0904010U0BIADAA,Barkat_G/F W/F Wte Rice Pizza Crust,1,9.48
35,2018-03-01,14L,CCG,NHS MANCHESTER CCG,0904010ACBKABAC,Glutafin_G/F W/F Fibre Flakes,3,54.36
93,2018-06-01,09G,CCG,NHS COASTAL WEST SUSSEX CCG,090401080BPAJBD,Barkat_G/F W/F Lasagne,1,2.78
95,2018-11-01,01F,CCG,NHS HALTON CCG,0904010Q0BJAAAQ,BiAlimenta_G/F Pasta Tubetti,1,11.20
107,2018-01-01,05A,CCG,NHS COVENTRY AND RUGBY CCG,0904010A0BKAEBF,Barkat_G/F W/F Crispbread,1,5.82
119,2018-01-01,06K,CCG,NHS EAST AND NORTH HERTFORDSHIRE CCG,090401080BFAMBD,Orgran_G/F W/F Rice & Corn Lasagne,1,2.91
131,2018-09-01,15F,CCG,NHS LEEDS CCG,090401070BGADAI,Innovative Soln_PureG/F W/F TapiocaFlour,1,6.75
141,2018-09-01,08G,CCG,NHS HILLINGDON CCG,0904010ACBKABAC,Glutafin_G/F W/F Fibre Flakes,2,7.01
144,2018-05-01,07G,CCG,NHS THURROCK CCG,0904010U0BIACAA,Barkat_G/F W/F Brown Rice Pizza Crust,1,18.95
154,2018-05-01,07P,CCG,NHS BRENT CCG,0904010Q0BJADAQ,BiAlimenta_G/F Pasta Penne,3,73.75


In [21]:
bl2018.sum()

totalitems    244708.00
totalcost    1798801.89
dtype: float64

In [22]:
### see impact since change of gf regs
bl_post_gfregs = df_blacklist.loc[(df_blacklist["month"]>="2018-12-01")]
bl_post_gfregs

,month,pct,org_type,name,bnf_code,bnf_name,totalitems,totalcost
2601,2018-12-01,15F,CCG,NHS LEEDS CCG,090401070BGAGAI,Innovative Soln_PureG/F W/F WteTeffFlour,1,4.75
3485,2018-12-01,02P,CCG,NHS BARNSLEY CCG,1310020N0BBABAB,Daktarin_Pdr 2%,1,2.75
4624,2019-01-01,07K,CCG,NHS WEST SUFFOLK CCG,090401080BFAMBD,Orgran_G/F W/F Rice & Corn Lasagne,1,8.72
5319,2019-01-01,05G,CCG,NHS NORTH STAFFORDSHIRE CCG,0901020N0BBAAAB,Neo-Cytamen_'1000' Inj 1mg/ml 1ml Amp,1,2.32
6232,2018-12-01,00D,CCG,"NHS DURHAM DALES, EASINGTON AND SEDGEFIELD CCG",090607000BBCJA0,Orovite_High-Pot B & C Vit Tab,1,2.08
6770,2018-12-01,04J,CCG,NHS NORTH DERBYSHIRE CCG,0901020N0BBAAAB,Neo-Cytamen_'1000' Inj 1mg/ml 1ml Amp,1,11.59
7005,2018-12-01,08N,CCG,NHS REDBRIDGE CCG,0901020N0BBAAAB,Neo-Cytamen_'1000' Inj 1mg/ml 1ml Amp,2,4.64
8415,2019-02-01,08X,CCG,NHS WANDSWORTH CCG,0407010H0BFAGAQ,Panadol ActiFast Solb_Tab 500mg,2,32.19
9755,2019-01-01,08R,CCG,NHS MERTON CCG,0407010H0BFAGAQ,Panadol ActiFast Solb_Tab 500mg,1,5.21
11002,2018-12-01,02R,CCG,NHS BRADFORD DISTRICTS CCG,0906050P0BCABAB,Vita-E_Gels 200u,1,4.74


In [30]:
bl_post_gfregs.groupby(['bnf_name'])['totalcost'].sum().sort_values(ascending = False)


bnf_name
Daktarin_Crm 2%                            14613.75
Panadol ActiFast Solb_Tab 500mg             1026.49
Neo-Cytamen_'1000' Inj 1mg/ml 1ml Amp        723.12
Juvela_G/F W/F Fusilli                       294.75
Glutafin_G/F W/F Spiral                      150.14
Vita-E_Gels 200u                             141.55
Finasteride_Tab 1mg                          117.48
Regaine For Men_Ex Strgh Foam Aero 5%        108.42
Orgran_G/F Self Raising Flour                 95.09
Clarityn_Allergy Tab 10mg                     94.69
Juvela_G/F W/F Spaghetti                      87.12
Cobalin-H_Inj 1mg/ml 1ml Amp                  81.25
Glutafin_G/F W/F Pizza Base                   79.26
Glutafin_G/F Fibre Fusilli                    68.86
Juvela_G/F Pizza Base                         65.23
BiAlimenta_G/F Potato Pasta Gnocchi           63.63
Juvela_G/F W/F Cornflakes                     58.12
Dermacolor_Fixier A/Spy 150ml                 57.38
Juvela_G/F W/F Fibre Penne                    55.29
Juv